In [3]:
%load_ext autoreload
%autoreload
import os, json
from types import SimpleNamespace
from experiment import run_model
from eval import calculate_stats
import pickle
from datetime import datetime 
import torch
from transformers import BertTokenizer
from sklearn.metrics import classification_report

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"

In [5]:
config_folder = "config/"

class obj(object):
    def __init__(self, d):
        for a, b in d.items():
            if isinstance(b, (list, tuple)):
               setattr(self, a, [obj(x) if isinstance(x, dict) else x for x in b])
            else:
               setattr(self, a, obj(b) if isinstance(b, dict) else b)

with open(os.path.join(config_folder, "config.json")) as f:
    config = obj(json.load(f))

In [6]:
train_inputs, train_label_ids, train_preds, train_loss, dev_inputs, dev_label_ids, dev_loss, dev_preds = run_model(config, device)

INFO:root:***** Running training *****
INFO:root:  Num examples = 61014
INFO:root:  Batch size = 12
INFO:root:  Num steps = 50840
Epoch:   0%|                                                                                    | 0/10 [00:00<?, ?it/s]

C:\ProgramData\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
Epoch:  10%|███████▎                                                                 | 1/10 [10:19<1:32:54, 619.39s/it]


so far epoch accuracy:  0.9408824204280984


Epoch:  20%|██████████████▌                                                          | 2/10 [20:34<1:22:26, 618.25s/it]


so far epoch accuracy:  0.9765299767266529


...

Epoch:  90%|█████████████████████████████████████████████████████████████████▋       | 9/10 [1:32:20<10:15, 615.27s/it]


so far epoch accuracy:  0.9868718654734979


Epoch: 100%|████████████████████████████████████████████████████████████████████████| 10/10 [1:42:35<00:00, 615.52s/it]


so far epoch accuracy:  0.9875766217589406
Training complete
Best val Acc: 0.987577


INFO:root:***** Running training *****
INFO:root:  Num examples = 15304
INFO:root:  Batch size = 12
INFO:root:  Num steps = 12750


In [7]:
train_inputs, train_label_ids, train_preds, train_loss, dev_inputs, dev_label_ids, dev_loss, dev_preds

(array([[  101,   152, 16844, ...,     0,     0,     0],
        [  101,  1109,   152, ...,     0,     0,     0],
        [  101,   152,  2064, ...,     0,     0,     0],
        ...,
        [  101,   155,  9025, ...,     0,     0,     0],
        [  101,  1153,  1145, ...,     0,     0,     0],
        [  101,   156,  2591, ...,     0,     0,     0]], dtype=int64),
 array([1, 0, 3, ..., 8, 8, 0], dtype=int64),
 array([1, 0, 3, ..., 8, 8, 0], dtype=int64),
 0.028790767224691097,
 array([[  101,  1489,   119, ...,     0,     0,     0],
        [  101,  1153,  1144, ...,     0,     0,     0],
        [  101, 12120,  1348, ...,     0,     0,     0],
        ...,
        [  101,   129,   119, ...,     0,     0,     0],
        [  101,   125,   119, ...,     0,     0,     0],
        [  101,   156,  2591, ...,     0,     0,     0]], dtype=int64),
 array([6, 8, 8, ..., 8, 8, 0], dtype=int64),
 0.10140008431627397,
 array([6, 8, 8, ..., 8, 8, 0], dtype=int64))

In [8]:
print(classification_report(dev_label_ids,dev_preds ))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95      1034
           1       0.97      0.97      0.97      1145
           2       0.94      0.96      0.95      1382
           3       0.94      0.96      0.95      1269
           4       0.96      0.94      0.95       136
           5       0.95      0.94      0.95       954
           6       0.97      0.97      0.97       933
           7       0.89      0.91      0.90       250
           8       1.00      1.00      1.00      8201

    accuracy                           0.98     15304
   macro avg       0.95      0.95      0.95     15304
weighted avg       0.98      0.98      0.98     15304



In [13]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)
missed_cases = []
for i in range(0,50):
    if dev_label_ids[i] !=  dev_preds[i]:
         missed_cases.append([ dev_preds[i],  dev_label_ids[i] , " ". join (tokenizer.convert_ids_to_tokens(train_inputs[i])) ])

#Save into a file
missed_cases_file = config.programsettings.REPORTS_DIR + "BIOBERT_fc_missedcases_" + str(datetime.now()).replace(":", "_").replace(".", "_") + ".pkl"
with open(missed_cases_file, "wb") as f:
    pickle.dump(missed_cases, f)  


In [15]:
import numpy as np
np.array(dev_label_ids).shape, np.array(dev_preds).shape, np.array(train_preds).shape, np.array(train_label_ids).shape

((15304,), (15304,), (61014,), (61014,))

In [16]:
%autoreload
train_mcc, train_f1_score, train_df_results, train_label_matches_df = calculate_stats(train_label_ids,train_preds )
dev_mcc, dev_f1_score, dev_df_results, dev_label_matches_df = calculate_stats(dev_label_ids,dev_preds )


 label: [1 0 3 ... 8 8 0]

 preds: [1 0 3 ... 8 8 0]

 label: [6 8 8 ... 8 8 0]

 preds: [6 8 8 ... 8 8 0]


In [17]:
all_experiment_results = []
all_experiment_results.append([str(config), train_loss, dev_loss, train_mcc, train_f1_score,dev_mcc,dev_f1_score, 
                               dev_label_ids, dev_preds,train_label_ids,train_preds  ])
all_experiment_results

[['<__main__.obj object at 0x0000022DF20EBD60>',
  0.028790767224691097,
  0.10140008431627397,
  0.9824371776156186,
  0.9806957642676125,
  0.9660141318266612,
  0.9540357540100197,
  array([6, 8, 8, ..., 8, 8, 0], dtype=int64),
  array([6, 8, 8, ..., 8, 8, 0], dtype=int64),
  array([1, 0, 3, ..., 8, 8, 0], dtype=int64),
  array([1, 0, 3, ..., 8, 8, 0], dtype=int64)]]

In [18]:
dev_label_matches_df

predicted
labels         matched           
ADE-Drug       False           22
               True           228
Dosage-Drug    False           24
               True           909
Duration-Drug  False            8
               True           128
Form-Drug      False           60
               True           894
Frequency-Drug False           55
               True          1214
Reason-Drug    False           46
               True           988
Route-Drug     False           38
               True          1107
Strength-Drug  False           62
               True          1320
no relation    False           39
               True          8162

##  Try with different configuration

### Just change model from BIOR to BERT Sequence

In [19]:
config.programsettings.MODEL_NAME = "BERT_Sequence"
config.programsettings.DEBUG_PRINT = 0

In [ ]:
train_label_ids, train_preds, train_loss, dev_label_ids, dev_loss, dev_preds = run_model(config, device)

INFO:root:***** Running training *****
INFO:root:  Num examples = 61014
INFO:root:  Batch size = 12
INFO:root:  Num steps = 50840
INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased.tar.gz from cache at cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c
INFO:pytorch_pretrained_bert.modeling:extracting archive file cache/a803ce83ca27fecf74c355673c434e51c265fb8a3e0e57ac62a80e38ba98d384.681017f415dfb33ec8d0e04fe51a619f3f01532ecea04edbfd48c5d160550d9c to temp dir C:\Users\pnarsina\AppData\Local\Temp\2\tmppzlzf8h4
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_v

C:\ProgramData\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


In [ ]:

train_mcc, train_f1_score, train_df_results, train_label_matches_df = calculate_stats(train_label_ids,train_preds )
dev_mcc, dev_f1_score, dev_df_results, dev_label_matches_df = calculate_stats(dev_label_ids,dev_preds)

In [ ]:
all_experiment_results.append([str(config), train_loss, dev_loss, train_mcc, train_f1_score,dev_mcc,dev_f1_score, 
                               dev_label_ids, dev_preds,train_label_ids,train_preds  ])
all_experiment_results

In [ ]:
dev_label_matches_df

In [ ]:

all_model_results_pickle_file = config.programsettings.REPORTS_DIR + "multi_model_experiment_results_" + str(datetime.now()).replace(":", "_").replace(".", "_") + ".pkl"
with open(all_model_results_pickle_file, "wb") as f:
    pickle.dump(all_experiment_results, f)  


In [ ]:
print(classification_report(dev_label_ids,dev_preds ))

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)
missed_cases = []
for i in range(0,50):
    if dev_label_ids[i] !=  dev_preds[i]:
        missed_cases.append([ dev_preds[i],  dev_label_ids[i] , " ". join (tokenizer.convert_ids_to_tokens(train_inputs[i]))])

#Save into a file
missed_cases_file = config.programsettings.REPORTS_DIR + "BERT_sequential_missedcases_" + str(datetime.now()).replace(":", "_").replace(".", "_") + ".pkl"
with open(missed_cases_file, "wb") as f:
    pickle.dump(missed_cases, f)  
